In [4]:
import torch
import os
import torch
import math
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl
from torch import Tensor
from joblib import dump, load
import pandas as pd 

def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out
 
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes, num_channels=3):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7)
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(nn.Conv2d(self.inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False), nn.BatchNorm2d(planes * block.expansion),)
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

class test_CNN:
    def __init__(self):
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    #trying on 6 different resnets for different epochs
    def resnet18(self, num_classes, channels=3):
        return ResNet(BasicBlock, [2, 2, 2, 2], num_classes, channels)

    def resnet34(self, num_classes, channels=3):
        return ResNet(BasicBlock, [3, 4, 6, 3], num_classes, channels)

    def train(self, epoch, model, dataloader, lfn, optimizer):
        for i in range(epoch):
            model.train()
            print(f'Now Entering Epoch {i + 1}')
            lss=0.0
            for batch_idx, (data, target) in enumerate(dataloader):
                # send to device
                data, target = data.to(self.device), target.to(self.device)
                optimizer.zero_grad()
                output = model(data)
                loss = lfn(output, target)
                loss.backward()
                optimizer.step()
                lss += loss.item()
                if batch_idx % 10 == 0:
                    print('Loss after batch %5d: %.10f' % (batch_idx + 1, lss / 10))
                    lss = 0.0
        return model

    def mainmethod(self, loadname, pre_trained_model = None):
        if pre_trained_model == None:
            model = self.resnet34(num_classes = 12, channels = 4)
        else:
            model = pre_trained_model
        model = model.to(self.device)

        loss_function = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        read_prep_data = load(loadname)
        traindata, trainy = read_prep_data[0], read_prep_data[1]
        train_dataloader = DataLoader(dataset=TensorDataset(Tensor(traindata), Tensor(trainy)), batch_size=16, shuffle=True, num_workers=2)
        model = self.train(epoch=1, model=model, dataloader=train_dataloader, lfn=loss_function, optimizer=optimizer)
        return model

In [5]:
# -*- coding: utf-8 -*-
"""CapstoneProject_SubmissionCode_NikhilKorlipara.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/14Vrop0cu_epWj_gQpv1sdADgDwD_-FiE
"""
class Submission:
    def submit(self, filename, modelname):
        outfile = 'Nikhil Korlipara, nk2742.csv'
        output_file = open(outfile, 'w')
        titles = ['ID', 'FINGER_POS_1', 'FINGER_POS_2', 'FINGER_POS_3', 'FINGER_POS_4', 'FINGER_POS_5', 'FINGER_POS_6',
                 'FINGER_POS_7', 'FINGER_POS_8', 'FINGER_POS_9', 'FINGER_POS_10', 'FINGER_POS_11', 'FINGER_POS_12']

        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        print(torch.cuda.is_available())
        torch.cuda.empty_cache()

        test_data = load(filename)
        rgb_data = test_data[0]
        file_ids = test_data[1]
        file_ids = [int(numeric_string) for numeric_string in file_ids]

        prediction = np.zeros(shape=(rgb_data.shape[0], 12))
        datasetest = TensorDataset(Tensor(rgb_data), Tensor(prediction))
        bsz = 16
        test_dataloader = DataLoader(dataset=datasetest, batch_size=bsz, shuffle=False, num_workers=2)

        model = torch.jit.load(modelname)
        model.eval()

        for batch_inx, (data, target) in enumerate(test_dataloader):
            data, target = data.to(device), target.to(device)
            testprediction = model(data)
            prediction[batch_inx*bsz:(batch_inx+1)*bsz, :] = testprediction.detach().cpu().numpy()
            if batch_inx % 10 == 0:
                print('Batch Index: ' + str(batch_inx))

        df = pd.concat([pd.DataFrame(file_ids), pd.DataFrame(prediction)], axis=1, names=titles)
        df.columns = titles
        df.to_csv(outfile, index=False)
        print("Written to csv file {}".format(outfile))
        print('Complete')
        return df

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())
torch.cuda.empty_cache()
cnn_model = test_CNN()
cnn_model_0 = cnn_model.mainmethod(loadname = 'preprocessed_train_data0.joblib', pre_trained_model = None)
model_scripted = torch.jit.script(cnn_model_0) # Export to TorchScript
model_scripted.save('res34_pretrained_model_0.pt')

#cnn_model_1 = cnn_model.mainmethod(loadname = 'preprocessed_train_data1.joblib', pre_trained_model = None)
#model_scripted = torch.jit.script(cnn_model_1) # Export to TorchScript
#model_scripted.save('res34_pretrained_model_0.pt')

#cnn_model_2 = cnn_model.mainmethod(loadname = 'preprocessed_train_data2.joblib', pre_trained_model = None)
#model_scripted = torch.jit.script(cnn_model_2) # Export to TorchScript
#model_scripted.save('res34_pretrained_model_2.pt')

cnn_model_combined = cnn_model.mainmethod(loadname = 'preprocessed_train_data1.joblib', pre_trained_model = cnn_model_0)
cnn_model_combined = cnn_model.mainmethod(loadname = 'preprocessed_train_data2.joblib', pre_trained_model = cnn_model_combined)

model_scripted = torch.jit.script(cnn_model_combined) # Export to TorchScript
model_scripted.save('res34_pretrained_combined_model.pt')

False
Now Entering Epoch 1
Loss after batch     1: 0.0562697113
Loss after batch    11: 0.0599551541
Loss after batch    21: 0.0041477865
Loss after batch    31: 0.0014663359
Loss after batch    41: 0.0006385723
Loss after batch    51: 0.0006611973
Loss after batch    61: 0.0006403856
Loss after batch    71: 0.0004896866
Loss after batch    81: 0.0003813461
Loss after batch    91: 0.0005204501
Loss after batch   101: 0.0003365924
Loss after batch   111: 0.0004184877
Loss after batch   121: 0.0007139581
Loss after batch   131: 0.0005066775
Loss after batch   141: 0.0009651308
Loss after batch   151: 0.0009558064
Loss after batch   161: 0.0006205831
Loss after batch   171: 0.0013670005
Loss after batch   181: 0.0011568433
Loss after batch   191: 0.0007913806
Loss after batch   201: 0.0008582915
Loss after batch   211: 0.0007084779
Now Entering Epoch 2
Loss after batch     1: 0.0001368232
Loss after batch    11: 0.0012332542
Loss after batch    21: 0.0008278305
Loss after batch    31: 0.0

Loss after batch   181: 0.0003402178
Loss after batch   191: 0.0002093557
Loss after batch   201: 0.0002014052
Loss after batch   211: 0.0002104756
Now Entering Epoch 11
Loss after batch     1: 0.0000316362
Loss after batch    11: 0.0002392519
Loss after batch    21: 0.0001037627
Loss after batch    31: 0.0002266235
Loss after batch    41: 0.0002695191
Loss after batch    51: 0.0003138396
Loss after batch    61: 0.0003043489
Loss after batch    71: 0.0002584174
Loss after batch    81: 0.0003048159
Loss after batch    91: 0.0003390186
Loss after batch   101: 0.0004314284
Loss after batch   111: 0.0003903114
Loss after batch   121: 0.0002323571
Loss after batch   131: 0.0002680709
Loss after batch   141: 0.0002876039
Loss after batch   151: 0.0001784844
Loss after batch   161: 0.0001773414
Loss after batch   171: 0.0003629697
Loss after batch   181: 0.0002520950
Loss after batch   191: 0.0002722403
Loss after batch   201: 0.0005054974
Loss after batch   211: 0.0004660388
Now Entering Epo

Loss after batch   141: 0.0001483192
Loss after batch   151: 0.0001912497
Loss after batch   161: 0.0002141182
Loss after batch   171: 0.0002907523
Loss after batch   181: 0.0002463141
Loss after batch   191: 0.0002405119
Loss after batch   201: 0.0001543956
Loss after batch   211: 0.0002043064
Now Entering Epoch 10
Loss after batch     1: 0.0000319853
Loss after batch    11: 0.0003146231
Loss after batch    21: 0.0002589160
Loss after batch    31: 0.0001882811
Loss after batch    41: 0.0001596248
Loss after batch    51: 0.0001758260
Loss after batch    61: 0.0001367337
Loss after batch    71: 0.0001567773
Loss after batch    81: 0.0001449529
Loss after batch    91: 0.0001594667
Loss after batch   101: 0.0001230834
Loss after batch   111: 0.0001036623
Loss after batch   121: 0.0001875462
Loss after batch   131: 0.0001794502
Loss after batch   141: 0.0001761312
Loss after batch   151: 0.0002020977
Loss after batch   161: 0.0002267340
Loss after batch   171: 0.0002909707
Loss after batch

In [6]:
sub = Submission()
df = sub.submit(filename = 'preprocessed_test_img0.joblib', modelname = 'res34_pretrained_model_0.pt')

False
Batch Index: 0
Batch Index: 10
Batch Index: 20
Batch Index: 30
Batch Index: 40
Batch Index: 50
Written to csv file Nikhil Korlipara, nk2742.csv
Complete
